### Fama and French Factor Model: Understanding the Factors ### 

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from functions import import_daily_financial_data
from portfolios_toolkit import calculate_analytics

### Size Factor ###

In [5]:
def build_returns_dataframe(
        tickers: list, 
        start_date='2015-01-01', 
        end_date='2025-01-01'
) -> pd.DataFrame:
    # DataFrame
    df_returns = pd.DataFrame()

    # Loop
    for ticker in tickers:
        df = import_daily_financial_data(ticker, start_date, end_date, returns=True)
        if not df.empty:
            df_returns[ticker] = df['returns']
            print(f'Data Ready for {ticker}')

    return df_returns

In [3]:
# Import tickers categorized by market capitalization
tickers_df = pd.read_csv('../additional_data/mktcap_companies.csv')
tickers_df.drop(columns='Unnamed: 0', inplace=True)

tickers_df

In [4]:
# Set a dictionary
company_sizes = {
    'mega': tickers_df['mega_companies'],
    'large': tickers_df['large_companies'],
    'mid': tickers_df['mid_companies'],
    'small': tickers_df['small_companies'],
    'micro': tickers_df['micro_companies'],
}

In [5]:
# Get the data
#returns_by_size = {}

#for size, tickers in company_sizes.items():
#    returns_by_size[size] = build_returns_dataframe(tickers, size)

In [6]:
# Create the Portfolios
#portfolios_df = pd.DataFrame({
#    f'{size}_portfolio': df.mean(axis=1, skipna=True)
#    for size, df in returns_by_size.items()
#})

In [7]:
portfolios_df = pd.read_csv(r'..\additional_data\mktcap_portfolios.csv')
portfolios_df.drop(columns='Unnamed: 0', inplace=True)

portfolios_df

In [8]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(portfolios_df.cumsum(), label=portfolios_df.columns, alpha=1)

# Config
plt.title('Cumulative Returns (Size Adjusted) Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [9]:
# Analytics Table

size_analytics_table = calculate_analytics(portfolios_df)

size_analytics_table

In [10]:
# Calculate a primitive SMB premium

SMB = (1/2*(portfolios_df['small_portfolio']+portfolios_df['micro_portfolio']) - 
       1/2*(portfolios_df['mega_portfolio']+portfolios_df['large_portfolio']))  

In [11]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(SMB.cumsum(), label='SMB Premium', alpha=1)

# Config
plt.title('Cumulative SMB Premium Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [12]:
# Annualized Returns
annualized_returns = portfolios_df.mean() * 252 * 100
annualized_returns.name = 'annualized_returns'

annualized_returns

In [13]:
# Annualized Volatility
annualized_volatility = portfolios_df.std() * np.sqrt(252) * 10
annualized_volatility.name = 'annualized_volatility'

annualized_volatility

In [25]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(annualized_returns.iloc[::-1], label='Annualized Returns', alpha=1, marker='o')

# Config
plt.title('Annualized Returns by Size')
plt.xlabel('Size')
plt.ylabel('Annualized Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [26]:
data = pd.DataFrame({
    'returns': annualized_returns.iloc[::-1],
    'volatility': annualized_volatility.iloc[::-1],
}, index=annualized_returns.iloc[::-1].index)

errors = data['volatility']

# Create the plot
fig, ax = plt.subplots(figsize=(8, 5))

ax.errorbar(
    x=data['returns'],                # x-values (betas)
    y=range(len(data)),               # y-positions
    xerr=errors,                      # confidence interval errors
    fmt='o',                          # circular markers for betas
    ecolor='gray',                    # color of the error bars
    capsize=5,                        # small caps on error bars
    elinewidth=2,                     # thickness of the error bars
    markeredgewidth=2                 # thickness of the circle edge
)

# Customize the plot
ax.set_yticks(range(len(data)))
ax.set_yticklabels(data.index)
ax.axvline(0, color='red', linestyle='--')  # reference line
ax.set_xlabel('Annualized Returns')
ax.set_title('Returns with Standard Errors')

plt.tight_layout()
plt.show()

### Value Factor ###

In [2]:
# Define a High Value Stocks List
high_value_tickers = [
    "XOM",  # ExxonMobil
    "CVX",  # Chevron
    "JPM",  # JPMorgan
    "BAC",  # Bank of America
    "WFC",  # Wells Fargo
    "PFE",  # Pfizer
    "MRK",  # Merck
    "VZ",   # Verizon
    "T",    # AT&T
    "MMM",  # 3M
    "C",    # Citigroup
    "F",    # Ford
    "GM",   # General Motors
    "INTC", # Intel
    "IBM",  # IBM
    "MO",   # Altria
    "GILD", # Gilead
    "PRU",  # Prudential
    "MET",  # MetLife
    "DUK",  # Duke Energy
]

In [3]:
# And a Low Value Stocks List
low_value_tickers = [
    "TSLA",   # Tesla
    "NVDA",   # NVIDIA
    "AAPL",   # Apple
    "AMZN",   # Amazon
    "MSFT",   # Microsoft
    "META",   # Meta
    "NFLX",   # Netflix
    "SNOW",   # Snowflake
    "UBER",   # Uber
    "RIVN",   # Rivian
    "PLTR",   # Palantir
    "SHOP",   # Shopify
    "CRWD",   # CrowdStrike
    "ZS",     # Zscaler
    "PANW",   # Palo Alto
    "SQ",     # Block (Square)
    "COIN",   # Coinbase
    "SMCI",   # Supermicro
    "ABNB",   # Airbnb
    "ROKU",   # Roku
]

In [6]:
# Get the Data
high_value_returns = build_returns_dataframe(high_value_tickers)

In [7]:
high_value_returns

In [9]:
# And the Portfolio
high_value_portfolio = high_value_returns.mean(axis=1)
high_value_portfolio.name = 'high_value_portfolio'

high_value_portfolio

In [10]:
# Get the Data
low_value_returns = build_returns_dataframe(low_value_tickers)

In [11]:
low_value_returns

In [12]:
# And the Portfolio
low_value_portfolio = low_value_returns.mean(axis=1)
low_value_portfolio.name = 'low_value_portfolio'

low_value_portfolio

In [13]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(low_value_portfolio.cumsum(), label='Low Value Portfolio', alpha=1)
plt.plot(high_value_portfolio.cumsum(), label='High Value Portfolio', alpha=1)

# Config
plt.title('Cumulative Returns (Value Adjusted) Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()

In [14]:
# Create Plot
HML = high_value_portfolio - low_value_portfolio

plt.figure(figsize=(10, 6))
plt.plot(HML.cumsum(), label='HML', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns')
plt.legend()
plt.grid()

# Show
plt.show()